In [ ]:
!git clone https://github.com/AlessiaCicca/ASKQE-Hallucination.git

Cloning into 'ASKQE-Hallucination'...
remote: Enumerating objects: 107, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 107 (delta 36), reused 62 (delta 22), pack-reused 0 (from 0)
Receiving objects: 100% (107/107), 42.99 KiB | 7.16 MiB/s, done.
Resolving deltas: 100% (36/36), done.


In [ ]:
!rm -rf /content/ASKQE-Hallucination/data

IN THE ORIGINAL PAPER THEY USE THIS STRUCTURE:

{"id": "Wikipedia_handpicked_5:1872", "en": "If school closures occur late relative to an outbreak, they are less effective and may not have any impact at all.", "es": "Si el cierre de escuelas se establece tarde con relación al brote, es menos eficaz y puede que no genere ningún impacto.", "perturbation": "alteration", "pert_es": "Si el cierre de escuelas se establece temprano con relación al brote, es menos eficaz y puede que no genere ningún impacto."}


HOWEVER IF WE WANT TO USE MULTIPLE MT MODEL WE NEED A DIFFERENT STRUCTURE

NEW DATASET

{"id": "ex1", "src": "Published reports from China and Italy suggest that risk factors for severe COVID-19 disease include underlying health conditions, but data describing underlying health conditions among U.S. COVID-19 patients have not yet been reported."}
{"id": "ex2", "src": "Additionally, in some cases, the reopening of schools after a period of closure has resulted in increased infection rates."}
{"id": "ex3", "src": "The Royal College of General Practitioners (RCGP) Research and Surveillance Centre (RSC) and Public Health England (PHE) have successfully worked together on the surveillance of influenza and other infectious diseases for over 50 years, including three previous pandemics."}


In [ ]:
import torch
import gc

# Libera TUTTA la memoria GPU
torch.cuda.empty_cache()
gc.collect()

# Verifica
print(f"GPU libera: {torch.cuda.memory_reserved(0) / 1e9:.2f} GB")

GPU libera: 0.00 GB


TRANSLATION

In [ ]:
import json
import os
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import gc

# =========================
# DEVICE
# =========================
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"[INFO] Using device: {device}")

# =========================
# MODELS (SOLO 2 MODELLI)
# =========================
MT_MODELS = {
    "mt1": "Helsinki-NLP/opus-mt-en-it",          # Standard EN→IT
    "mt2": "Helsinki-NLP/opus-mt-tc-big-en-it"   # Big EN→IT
}

# =========================
# LOAD MODEL
# =========================
def load_model(model_name):
    print(f"[LOAD] {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
    return tokenizer, model

def free_model(model, tokenizer):
    if model is not None:
        del model
    if tokenizer is not None:
        del tokenizer
    torch.cuda.empty_cache()
    gc.collect()

# =========================
# TRANSLATION FUNCTION
# =========================
def translate_marian(text, tokenizer, model):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    ).to(device)

    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=256)

    return tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

# =========================
# LOAD DATA
# =========================
INPUT_PATH = "/content/ASKQE-Hallucination/data.json"   # json o jsonl
OUTPUT_PATH = "/content/ASKQE-Hallucination/data/src_mt.jsonl"

os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

def load_data(path):
    print(f"[STEP] Loading dataset: {path}")
    with open(path, "r", encoding="utf-8") as f:
        first_char = f.read(1)
        f.seek(0)
        if first_char == "[":
            data = json.load(f)
            print(f"[OK] JSON list with {len(data)} samples")
            return data
        else:
            data = [json.loads(line) for line in f if line.strip()]
            print(f"[OK] JSONL with {len(data)} samples")
            return data

data_list = load_data(INPUT_PATH)

# =========================
# PHASE 1: MT (EN → IT)
# =========================
print("\n" + "="*70)
print("PHASE 1: MACHINE TRANSLATION (EN → IT) - 2 MODELS")
print("="*70 + "\n")

for mt_key, mt_model_name in MT_MODELS.items():
    print(f"\n[MT] Loading {mt_key}: {mt_model_name.split('/')[-1]}")
    mt_tok, mt_model = load_model(mt_model_name)

    for idx, ex in enumerate(data_list, start=1):
        src = ex.get("src", None)
        if not src:
            continue

        ex[mt_key] = translate_marian(src, mt_tok, mt_model)
        print(f"  [{idx}/{len(data_list)}] {mt_key}: {ex[mt_key][:70]}...")

    print(f"[CLEANUP] Freeing {mt_key}")
    free_model(mt_model, mt_tok)

# =========================
# SAVE
# =========================
print("\n" + "="*70)
print("SAVING RESULTS")
print("="*70 + "\n")

with open(OUTPUT_PATH, "w", encoding="utf-8") as fout:
    for ex in data_list:
        fout.write(json.dumps(ex, ensure_ascii=False) + "\n")

print("✅ PIPELINE COMPLETED SUCCESSFULLY!")
print(f"📄 Output saved to: {OUTPUT_PATH}")
print(f"📊 Total samples: {len(data_list)}")
print("\nModels used:")
print("  MT1: Helsinki-NLP/opus-mt-en-it")
print("  MT2: Helsinki-NLP/opus-mt-tc-big-en-it")


[INFO] Using device: cuda
[STEP] Loading dataset: /content/ASKQE-Hallucination/data.json
[OK] JSONL with 3 samples

PHASE 1: MACHINE TRANSLATION (EN → IT) - 2 MODELS


[MT] Loading mt1: opus-mt-en-it
[LOAD] Helsinki-NLP/opus-mt-en-it


/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


  [1/3] mt1: I rapporti pubblicati dalla Cina e dall'Italia suggeriscono che i fatt...
  [2/3] mt1: Inoltre, in alcuni casi, la riapertura delle scuole dopo un periodo di...
  [3/3] mt1: La chiusura delle scuole nella città di Oita, in Giappone, è risultata...
[CLEANUP] Freeing mt1

[MT] Loading mt2: opus-mt-tc-big-en-it
[LOAD] Helsinki-NLP/opus-mt-tc-big-en-it
  [1/3] mt2: I rapporti pubblicati dalla Cina e dall’Italia suggeriscono che i fatt...
  [2/3] mt2: Inoltre, in alcuni casi, la riapertura delle scuole dopo un periodo di...
  [3/3] mt2: Le chiusure delle scuole nella città di Oita, in Giappone, hanno ridot...
[CLEANUP] Freeing mt2

SAVING RESULTS

✅ PIPELINE COMPLETED SUCCESSFULLY!
📄 Output saved to: /content/ASKQE-Hallucination/data/src_mt.jsonl
📊 Total samples: 3

Models used:
  MT1: Helsinki-NLP/opus-mt-en-it
  MT2: Helsinki-NLP/opus-mt-tc-big-en-it


In [ ]:
import json
import os
import torch
import gc
import random
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    MarianMTModel,
    MarianTokenizer
)

# =========================
# DEVICE
# =========================
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"[INFO] Using device: {device}")

# =========================
# WORST MT MODELS (BUT ITALIAN OUTPUT)
# =========================
MT_MODELS = {
    # Tiny, generic Romance → Italian included (very noisy)
    "mt1": "Helsinki-NLP/opus-mt-en-roa",

    # Multilingual confused model (Italian forced)
    "mt2": "facebook/mbart-large-50-one-to-many-mmt"
}

# =========================
# LOAD MODEL
# =========================
def load_model(model_name):
    print(f"[LOAD] {model_name}")
    try:
        if "opus-mt" in model_name or "Helsinki" in model_name:
            tokenizer = MarianTokenizer.from_pretrained(model_name)
            model = MarianMTModel.from_pretrained(model_name).to(device)
            model.eval()
            return tokenizer, model, "marian"
        else:
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
            model.eval()
            return tokenizer, model, "mbart"
    except Exception as e:
        print(f"[ERROR] Cannot load {model_name}: {e}")
        return None, None, None

def free_model(model, tokenizer):
    del model
    del tokenizer
    torch.cuda.empty_cache()
    gc.collect()

# =========================
# EXTREME HALLUCINATION TRANSLATION (ITALIAN OUTPUT)
# =========================
def translate_worst_quality(text, tokenizer, model, model_type):
    if model_type == "marian":
        inputs = tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            max_length=256
        ).to(device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                do_sample=True,
                temperature=3.0,
                top_p=0.5,
                top_k=15,
                num_beams=1,
                max_new_tokens=200,
                repetition_penalty=0.5,
                length_penalty=0.3,
                no_repeat_ngram_size=0,
                early_stopping=False
            )

    else:
        # mBART: force EN → IT
        tokenizer.src_lang = "en_XX"
        forced_bos_token_id = tokenizer.lang_code_to_id["it_IT"]

        inputs = tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            max_length=256
        ).to(device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                do_sample=True,
                temperature=3.0,
                top_p=0.5,
                top_k=15,
                num_beams=1,
                max_new_tokens=200,
                repetition_penalty=0.5,
                length_penalty=0.3,
                forced_bos_token_id=forced_bos_token_id,
                early_stopping=False
            )

    return tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

# =========================
# INPUT NOISE (OPTIONAL)
# =========================
def add_noise_to_text(text, noise_level=0.15):
    words = text.split()
    n = max(1, int(len(words) * noise_level))

    for _ in range(n):
        i = random.randint(0, len(words) - 1)
        op = random.choice(["duplicate", "remove"])
        if op == "duplicate":
            words.insert(i, words[i])
        elif op == "remove" and len(words) > 3:
            words.pop(i)

    return " ".join(words)

# =========================
# LOAD DATA
# =========================
INPUT_PATH = "/content/ASKQE-Hallucination/data.json"
OUTPUT_PATH = "/content/ASKQE-Hallucination/data/src_mt_WORST.jsonl"
os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

with open(INPUT_PATH, "r", encoding="utf-8") as f:
    data_list = json.load(f)

# =========================
# RUN MT
# =========================
for mt_key, model_name in MT_MODELS.items():
    print(f"\n[RUN] {mt_key} → {model_name}")
    tok, mdl, mtype = load_model(model_name)

    for i, ex in enumerate(data_list, 1):
        src = ex.get("src")
        if not src:
            continue

        noisy_src = add_noise_to_text(src) if mt_key == "mt1" else src
        ex[mt_key] = translate_worst_quality(noisy_src, tok, mdl, mtype)

        print(f"  [{i}/{len(data_list)}] {mt_key}: {ex[mt_key][:80]}...")

    free_model(mdl, tok)

# =========================
# SAVE
# =========================
with open(OUTPUT_PATH, "w", encoding="utf-8") as f:
    for ex in data_list:
        f.write(json.dumps(ex, ensure_ascii=False) + "\n")

print("\n✅ WORST MT PIPELINE COMPLETED (ITALIAN OUTPUT GUARANTEED)")
print(f"📄 Output: {OUTPUT_PATH}")


[INFO] Using device: cuda


JSONDecodeError: Extra data: line 2 column 1 (char 261)

OUTPUT TYPE:

{"id": "ex3", "src": "The agreement will remain valid until December 31, 2026, unless terminated earlier by either party.", "mt1": "L'accordo resterà valido fino al 31 dicembre 2026, a meno che non sia stato risolto prima da una delle parti.", "mt2": "L'accordo rimarrà valido fino al 31 dicembre 2026, a meno che non sia stato risolto in precedenza da una delle parti."}



PERTUBATION ON THE TRANSLATION

In [ ]:
!python -u /content/ASKQE-Hallucination/perturbMistral.py \
  --input_path /content/ASKQE-Hallucination/data/src_mt.jsonl \
  --output_path /content/ASKQE-Hallucination/data/src_mt_perturb.jsonl \
  --perturbation_type entity_injection

[INFO] Using device: cuda
`torch_dtype` is deprecated! Use `dtype` instead!
E0000 00:00:1766337369.011741   33507 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766337369.115606   33507 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766337369.891198   33507 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766337369.891246   33507 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766337369.891251   33507 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766337369.891254   33507 computat

In [ ]:
!python -u /content/ASKQE-Hallucination/perturbMistral.py \
  --input_path /content/ASKQE-Hallucination/data/src_mt.jsonl \
  --output_path /content/ASKQE-Hallucination/data/src_mt_perturb.jsonl \
  --perturbation_type numerical_injection

[INFO] Using device: cuda
`torch_dtype` is deprecated! Use `dtype` instead!
E0000 00:00:1766337824.928831   35482 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766337824.991926   35482 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766337825.483789   35482 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766337825.483828   35482 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766337825.483833   35482 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766337825.483837   35482 computat

In [ ]:
!python -u /content/ASKQE-Hallucination/perturbMistral.py \
  --input_path /content/ASKQE-Hallucination/data/src_mt.jsonl \
  --output_path /content/ASKQE-Hallucination/data/src_mt_perturb.jsonl \
  --perturbation_type over_specification

[INFO] Using device: cuda
`torch_dtype` is deprecated! Use `dtype` instead!
E0000 00:00:1766338162.312562   36869 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766338162.321880   36869 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766338162.353988   36869 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766338162.354018   36869 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766338162.354024   36869 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766338162.354028   36869 computat

OUTPYT TYPE:

{"id": "ex3", "src": "The agreement will remain valid until December 31, 2026, unless terminated earlier by either party.", "mt1": "L'accordo resterà valido fino al 31 dicembre 2026, a meno che non sia stato risolto prima da una delle parti.", "mt2": "L'accordo rimarrà valido fino al 31 dicembre 2026, a meno che non sia stato risolto in precedenza da una delle parti.", "pert_mt1": "L'accordo resterà valido fino al 31 dicembre 2026, a meno che non sia stato risolto prima da una delle parti. Questa data può essere estesa in caso di accordi aggiuntivi fino al 31 dicembre 2027.", "pert_mt2": "L'accordo rimarrà valido fino al 31 dicembre 2026, a meno che non sia stato risolto in precedenza da una delle parti. Il prossimo aggiornamento sarà previsto per il 31 dicembre 2027."}


BACKTRANSLATION

In [ ]:
import json
import os
import torch
from transformers import MarianMTModel, MarianTokenizer
import gc
import re

# =========================
# DEVICE
# =========================
default_device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"[INFO] Using device: {default_device}")

# =========================
# MODELS (SOLO 2 MODELLI)
# =========================

BT_MODELS = {
    "bt1": "Helsinki-NLP/opus-mt-it-en",          # Standard IT→EN
    "bt2": "Helsinki-NLP/opus-mt-tc-big-it-en"   # Big IT→EN
}

# =========================
# LOAD MODEL
# =========================
def load_model(model_name):
    print(f"[LOAD] {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
    return tokenizer, model

def free_model(model, tokenizer):
    if model is not None:
        del model
    if tokenizer is not None:
        del tokenizer
    torch.cuda.empty_cache()
    gc.collect()

# =========================
# TRANSLATION FUNCTION
# =========================
def translate_marian(text, tokenizer, model):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    ).to(device)

    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=256)

    return tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

# =========================
# LOAD DATA
# =========================
INPUT_PATH = "/content/ASKQE-Hallucination/data/src_mt_perturb.jsonl"   # json o jsonl
OUTPUT_PATH = "/content/ASKQE-Hallucination/data/src_mt_perturb_bt.jsonl"

os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

def load_data(path):
    print(f"[STEP] Loading dataset: {path}")
    with open(path, "r", encoding="utf-8") as f:
        first_char = f.read(1)
        f.seek(0)
        if first_char == "[":
            data = json.load(f)
            print(f"[OK] JSON list with {len(data)} samples")
            return data
        else:
            data = [json.loads(line) for line in f if line.strip()]
            print(f"[OK] JSONL with {len(data)} samples")
            return data

data_list = load_data(INPUT_PATH)


# =========================
# PHASE 2: BACK-TRANSLATION (IT → EN)
# =========================
print("\n" + "="*70)
print("PHASE 2: BACK-TRANSLATION (IT → EN) - 2 MODELS × 2 MT = 4")
print("="*70 + "\n")
for bt_key_outer, bt_model_name_outer in BT_MODELS.items():
    print(f"\n[BT] Loading {bt_key_outer}: {bt_model_name_outer.split('/')[-1]}")
    bt_tok_outer, bt_model_outer = load_model(bt_model_name_outer)

    # bt1 → pert_mt1, bt2 → pert_mt2
    bt_index = bt_key_outer.replace("bt", "")
    pert_key = f"pert_mt{bt_index}"

    for idx, ex in enumerate(data_list, start=1):
        if pert_key not in ex:
            continue

        input_text = ex[pert_key]

        # ✅ output SOLO bt1 / bt2
        ex[bt_key_outer] = translate_marian(
            input_text,
            bt_tok_outer,
            bt_model_outer
        )

        print(f"  [{idx}/{len(data_list)}] {bt_key_outer}: {ex[bt_key_outer][:55]}...")

    print(f"[CLEANUP] Freeing {bt_key_outer}")
    free_model(bt_model_outer, bt_tok_outer)


# =========================
# SAVE
# =========================
print("\n" + "="*70)
print("SAVING RESULTS")
print("="*70 + "\n")

with open(OUTPUT_PATH, "w", encoding="utf-8") as fout:
    for ex in data_list:
        fout.write(json.dumps(ex, ensure_ascii=False) + "\n")

print("✅ PIPELINE COMPLETED SUCCESSFULLY!")
print(f"📄 Output saved to: {OUTPUT_PATH}")
print(f"📊 Total samples: {len(data_list)}")
print("📋 Fields per sample: src + 2 MT (IT) + 4 BT (EN) = 7 fields total")
print("\nModels used:")
print("  BT1: Helsinki-NLP/opus-mt-it-en")
print("  BT2: Helsinki-NLP/opus-mt-tc-big-it-en")


[INFO] Using device: cuda
[STEP] Loading dataset: /content/ASKQE-Hallucination/data/src_mt_perturb.jsonl
[OK] JSONL with 9 samples

PHASE 2: BACK-TRANSLATION (IT → EN) - 2 MODELS × 2 MT = 4


[BT] Loading bt1: opus-mt-it-en
[LOAD] Helsinki-NLP/opus-mt-it-en


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

source.spm:   0%|          | 0.00/814k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/790k [00:00<?, ?B/s]

...

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/576M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/576M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

  [1/9] bt2: Reports published by China and Italy suggest that risk ...
  [2/9] bt2: In addition, in some cases, the reopening of schools af...
  [3/9] bt2: School closures in the city of Oita, Japan have success...
  [4/9] bt2: Reports published by China and Italy suggest that risk ...
  [5/9] bt2: In addition, in some cases, the reopening of schools af...
  [6/9] bt2: School closures in the city of Oita, Japan have success...
  [7/9] bt2: Reports published by China and Italy suggest that risk ...
  [8/9] bt2: In addition, in some cases, the reopening of schools af...
  [9/9] bt2: School closures in the city of Oita, Japan have sharply...
[CLEANUP] Freeing bt2

SAVING RESULTS

✅ PIPELINE COMPLETED SUCCESSFULLY!
📄 Output saved to: /content/ASKQE-Hallucination/data/src_mt_perturb_bt.jsonl
📊 Total samples: 9
📋 Fields per sample: src + 2 MT (IT) + 4 BT (EN) = 7 fields total

Models used:
  BT1: Helsinki-NLP/opus-mt-it-en
  BT2: Helsinki-NLP/opus-mt-tc-big-it-en


OUTPUT TYPE:

{"id": "ex3", "src": "The agreement will remain valid until December 31, 2026, unless terminated earlier by either party.", "mt1": "L'accordo resterà valido fino al 31 dicembre 2026, a meno che non sia stato risolto prima da una delle parti.", "mt2": "L'accordo rimarrà valido fino al 31 dicembre 2026, a meno che non sia stato risolto in precedenza da una delle parti.", "pert_mt1": "L'accordo resterà valido fino al 31 dicembre 2026, a meno che non sia stato risolto prima da una delle parti. Inoltre, sarà valente per un totale di 7 anni.", "pert_mt2": "L'accordo rimarrà valido fino al 31 dicembre 2026, a meno che non sia stato risolto in precedenza da una delle parti. Il progetto sarà sospeso temporaneamente per due settimane in caso di emergenza sanitaria.", "bt1": "The agreement will remain valid until 31 December 2026, unless it has been previously resolved by one of the parties. The project will be temporarily suspended for two weeks in the event of a health emergency.", "bt2": "The agreement will remain valid until 31 December 2026, unless it has been previously resolved by one of the parties. The project will be temporarily suspended for two weeks in case of a health emergency."}



In [ ]:
!python -u /content/ASKQE-Hallucination/QG/llama-8b.py \
  --output_path /content/ASKQE-Hallucination/data/src_mt_perturb_bt_qg.jsonl \
  --prompt vanilla

`torch_dtype` is deprecated! Use `dtype` instead!
2025-12-21 18:15:56.447358: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766340956.782407   48373 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766340956.863785   48373 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766340957.511645   48373 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766340957.511685   48373 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766340957.511690   48373

OUTPUT TYPE

{"id": "ex3", "src": "The agreement will remain valid until December 31, 2026, unless terminated earlier by either party.", "mt1": "L'accordo resterà valido fino al 31 dicembre 2026, a meno che non sia stato risolto prima da una delle parti.", "mt2": "L'accordo rimarrà valido fino al 31 dicembre 2026, a meno che non sia stato risolto in precedenza da una delle parti.", "pert_mt1": "L'accordo resterà valido fino al 31 dicembre 2026, a meno che non sia stato risolto prima da una delle parti. Inoltre, sarà valente per un totale di 7 anni.", "pert_mt2": "L'accordo rimarrà valido fino al 31 dicembre 2026, a meno che non sia stato risolto in precedenza da una delle parti. Il progetto sarà sospeso temporaneamente per due settimane in caso di emergenza sanitaria.", "bt1": "The agreement will remain valid until 31 December 2026, unless it has been previously resolved by one of the parties. The project will be temporarily suspended for two weeks in the event of a health emergency.", "bt2": "The agreement will remain valid until 31 December 2026, unless it has been previously resolved by one of the parties. The project will be temporarily suspended for two weeks in case of a health emergency.", "questions_bt1": "[\"Until what date will the agreement remain valid?\", \"What can terminate the agreement?\", \"What happens if the agreement is not terminated before 31 December", "questions_bt2": "[\"When will the agreement no longer be valid?\", \"What can resolve the agreement?\", \"For how long will the agreement remain valid?\", \"What event will cause"}


In [ ]:
#QA su SRC
!python ASKQE-Hallucination/QA/llama-8b.py \
  --sentence_type src \
  --input_path /content/ASKQE-Hallucination/data/src_mt_perturb_bt_qg.jsonl \
  --output_path /content/ASKQE-Hallucination/data/qa_src.jsonl


[INFO] Using device: cuda
`torch_dtype` is deprecated! Use `dtype` instead!
2025-12-21 18:23:56.464007: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766341436.489493   50535 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766341436.497742   50535 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766341436.525689   50535 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766341436.525721   50535 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00

OUTPUT TYPE:

{"id": "ex3", "src": "The agreement will remain valid until December 31, 2026, unless terminated earlier by either party.", "mt1": "L'accordo resterà valido fino al 31 dicembre 2026, a meno che non sia stato risolto prima da una delle parti.", "mt2": "L'accordo rimarrà valido fino al 31 dicembre 2026, a meno che non sia stato risolto in precedenza da una delle parti.", "pert_mt1": "L'accordo resterà valido fino al 31 dicembre 2026, a meno che non sia stato risolto prima da una delle parti. Inoltre, sarà valente per un totale di 7 anni.", "pert_mt2": "L'accordo rimarrà valido fino al 31 dicembre 2026, a meno che non sia stato risolto in precedenza da una delle parti. Il progetto sarà sospeso temporaneamente per due settimane in caso di emergenza sanitaria.", "bt1": "The agreement will remain valid until 31 December 2026, unless it has been previously resolved by one of the parties. The project will be temporarily suspended for two weeks in the event of a health emergency.", "bt2": "The agreement will remain valid until 31 December 2026, unless it has been previously resolved by one of the parties. The project will be temporarily suspended for two weeks in case of a health emergency.", "questions_bt1": "[\"Until what date will the agreement remain valid?\", \"What can terminate the agreement?\", \"What happens if the agreement is not terminated before 31 December", "questions_bt2": "[\"When will the agreement no longer be valid?\", \"What can resolve the agreement?\", \"For how long will the agreement remain valid?\", \"What event will cause", "answers_src_bt1": ["[\"December 31, 2026\", \"either party\", \"the agreement remains valid\"]"], "answers_src_bt2": ["[\"December 31, 2026\", \"be terminated\", \"until December 31, 2026\", \"the agreement be terminated\"]"]}


In [ ]:
# QA su BT
!python ASKQE-Hallucination/QA/llama-8b.py \
  --sentence_type bt \
  --input_path /content/ASKQE-Hallucination/data/src_mt_perturb_bt_qg.jsonl \
  --output_path /content/ASKQE-Hallucination/data/qa_bt.jsonl

[INFO] Using device: cuda
`torch_dtype` is deprecated! Use `dtype` instead!
2025-12-21 18:26:00.222437: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766341560.249168   51087 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766341560.256565   51087 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766341560.285808   51087 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766341560.285836   51087 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

OUTPUT TYPE

{"id": "ex3", "src": "The agreement will remain valid until December 31, 2026, unless terminated earlier by either party.", "mt1": "L'accordo resterà valido fino al 31 dicembre 2026, a meno che non sia stato risolto prima da una delle parti.", "mt2": "L'accordo rimarrà valido fino al 31 dicembre 2026, a meno che non sia stato risolto in precedenza da una delle parti.", "pert_mt1": "L'accordo resterà valido fino al 31 dicembre 2026, a meno che non sia stato risolto prima da una delle parti. Inoltre, sarà valente per un totale di 7 anni.", "pert_mt2": "L'accordo rimarrà valido fino al 31 dicembre 2026, a meno che non sia stato risolto in precedenza da una delle parti. Il progetto sarà sospeso temporaneamente per due settimane in caso di emergenza sanitaria.", "bt1": "The agreement will remain valid until 31 December 2026, unless it has been previously resolved by one of the parties. The project will be temporarily suspended for two weeks in the event of a health emergency.", "bt2": "The agreement will remain valid until 31 December 2026, unless it has been previously resolved by one of the parties. The project will be temporarily suspended for two weeks in case of a health emergency.", "questions_bt1": "[\"Until what date will the agreement remain valid?\", \"What can terminate the agreement?\", \"What happens if the agreement is not terminated before 31 December", "questions_bt2": "[\"When will the agreement no longer be valid?\", \"What can resolve the agreement?\", \"For how long will the agreement remain valid?\", \"What event will cause", "answers_bt1": ["[\"31 December 2026\", \"it has been previously resolved by one of the parties\", \"No answer\"]"], "answers_bt2": ["[\"31 December 2026\", \"One of the parties\", \"Until 31 December 2026\", \"a health emergency\"]"]}


In [ ]:
import json
import re

def ucr_hallucination(answers_src, answers_bt, no_answer="No Answer"):
    assert len(answers_src) == len(answers_bt), f"Lengths mismatch: src={len(answers_src)}, bt={len(answers_bt)}"

    hallucinated = 0
    for a_src, a_bt in zip(answers_src, answers_bt):
        if a_src.strip() == no_answer and a_bt.strip() != no_answer:
            hallucinated += 1
    return hallucinated / max(1, len(answers_src))


INPUT_PATH_src = "/content/ASKQE-Hallucination/data/qa_src.jsonl"
INPUT_PATH_bt = "/content/ASKQE-Hallucination/data/qa_bt.jsonl"


def load_data(path):
    print(f"[STEP] Loading dataset: {path}")
    with open(path, "r", encoding="utf-8") as f:
        first_char = f.read(1)
        f.seek(0)
        if first_char == "[":
            data = json.load(f)
            print(f"[OK] JSON list with {len(data)} samples")
            return data
        else:
            data = [json.loads(line) for line in f if line.strip()]
            print(f"[OK] JSONL with {len(data)} samples")
            return data

src = load_data(INPUT_PATH_src)
bt = load_data(INPUT_PATH_bt)

# Define the default "No Answer" string
DEFAULT_NO_ANSWER = "No Answer"
DEFAULT_QUESTIONS_PER_ITEM = 3 # Assuming 3 questions per item if parsing fails

# Helper function to get the number of questions, more robust to truncation/malformation
def get_num_questions(questions_str, item_id):
    if not questions_str:
        return DEFAULT_QUESTIONS_PER_ITEM # Assume default if string is empty
    try:
        parsed_questions = json.loads(questions_str)
        return len(parsed_questions)
    except json.JSONDecodeError:
        print(f"Warning: Could not parse questions string for ID {item_id}: '{questions_str[:100]}...'. Assuming {DEFAULT_QUESTIONS_PER_ITEM} questions.")
        return DEFAULT_QUESTIONS_PER_ITEM # Assume default if parsing fails


# Aggregate all answers into flat lists
all_answers_src_bt1 = []
all_answers_bt1 = []

for src_item, bt_item in zip(src, bt):
    item_id = src_item.get('id', 'N/A')

    # Determine expected number of questions from the SRC item's questions_bt1
    # This will be the common reference for both src and bt answer lists
    num_questions_reference = get_num_questions(src_item.get("questions_bt1", ""), item_id)

    # Process answers_src_bt1
    if "answers_src_bt1" in src_item and src_item["answers_src_bt1"]:
        ans_list_str = src_item["answers_src_bt1"][0] # Expecting "["ans1", "ans2"]" as a string
        try:
            parsed_answers = json.loads(ans_list_str)
            # Adjust parsed_answers length to match the reference number of questions
            if len(parsed_answers) < num_questions_reference:
                parsed_answers.extend([DEFAULT_NO_ANSWER] * (num_questions_reference - len(parsed_answers)))
            elif len(parsed_answers) > num_questions_reference:
                print(f"Warning: Too many answers parsed for answers_src_bt1 for ID {item_id}. Truncating from {len(parsed_answers)} to {num_questions_reference} answers.")
                parsed_answers = parsed_answers[:num_questions_reference]
            all_answers_src_bt1.extend(parsed_answers)

        except json.JSONDecodeError:
            print(f"Warning: Could not parse answers_src_bt1 string '{ans_list_str[:100]}...' for ID {item_id}. Appending '{DEFAULT_NO_ANSWER}' for {num_questions_reference} questions.")
            all_answers_src_bt1.extend([DEFAULT_NO_ANSWER] * num_questions_reference)
    else:
        # If answers_src_bt1 is missing or empty, append "No Answer" for each question
        all_answers_src_bt1.extend([DEFAULT_NO_ANSWER] * num_questions_reference)


    # Process answers_bt1
    if "answers_bt1" in bt_item and bt_item["answers_bt1"]:
        ans_list_str = bt_item["answers_bt1"][0] # Expecting "["ans1", "ans2"]" as a string
        try:
            parsed_answers = json.loads(ans_list_str)
            # Adjust parsed_answers length to match the reference number of questions
            if len(parsed_answers) < num_questions_reference:
                parsed_answers.extend([DEFAULT_NO_ANSWER] * (num_questions_reference - len(parsed_answers)))
            elif len(parsed_answers) > num_questions_reference:
                print(f"Warning: Too many answers parsed for answers_bt1 for ID {item_id}. Truncating from {len(parsed_answers)} to {num_questions_reference} answers.")
                parsed_answers = parsed_answers[:num_questions_reference]
            all_answers_bt1.extend(parsed_answers)

        except json.JSONDecodeError:
            print(f"Warning: Could not parse answers_bt1 string '{ans_list_str[:100]}...' for ID {item_id}. Appending '{DEFAULT_NO_ANSWER}' for {num_questions_reference} questions.")
            all_answers_bt1.extend([DEFAULT_NO_ANSWER] * num_questions_reference)
    else:
        # If answers_bt1 is missing or empty, append "No Answer" for each question
        all_answers_bt1.extend([DEFAULT_NO_ANSWER] * num_questions_reference)

ucr_bt1 = ucr_hallucination(
    all_answers_src_bt1,
    all_answers_bt1,
    no_answer=DEFAULT_NO_ANSWER
)
print(ucr_bt1)


[STEP] Loading dataset: /content/ASKQE-Hallucination/data/qa_src.jsonl
[OK] JSONL with 9 samples
[STEP] Loading dataset: /content/ASKQE-Hallucination/data/qa_bt.jsonl
[OK] JSONL with 9 samples
0.1111111111111111


DA

"L'azienda ha registrato un utile netto di 2 milioni di dollari nel terzo trimestre."

A

 "L'azienda ha registrato un utile netto di 2 milioni di dollari nel terzo trimestre e un debito corrente di 15 milioni di dollari."

 DA

 "Il paziente è stato ricoverato in ospedale a causa di un forte dolore al torace che si è irradiato al braccio sinistro."

 A

"Il paziente è stato ricoverato in ospedale a causa di un forte dolore al torace che si è irradiato al braccio sinistro e ha subito una perdita di sensibilità nella mano destra pari al 20%.






---



In [ ]:
%cd /content/ASKQE-Hallucination

/content/ASKQE-Hallucination


In [ ]:
!git status


On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   QA/__pycache__/prompt.cpython-312.pyc
	modified:   QG/__pycache__/prompt.cpython-312.pyc
	modified:   data.json
	modified:   data/qa_bt.jsonl
	modified:   data/qa_src.jsonl
	modified:   data/src_mt.jsonl
	modified:   data/src_mt_perturb.jsonl
	modified:   data/src_mt_perturb_bt.jsonl
	modified:   data/src_mt_perturb_bt_qg.jsonl

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	perturbMistral.py

no changes added to commit (use "git add" and/or "git commit -a")


In [26]:
!git add .


In [ ]:
!git config --global user.email "alessia.ciccaglione02@gmail.com"
!git config --global user.name "AlessiaCicca"


In [29]:
!git commit -m "Traduzione con LLM no pertubation"


[main 6a4861a] Traduzione con LLM no pertubation
 17 files changed, 113 insertions(+), 50 deletions(-)
 delete mode 100644 data/qa_bt.jsonl
 delete mode 100644 data/qa_src.jsonl
 delete mode 100644 data/src_mt.jsonl
 delete mode 100644 data/src_mt_perturb.jsonl
 delete mode 100644 data/src_mt_perturb_bt.jsonl
 delete mode 100644 data/src_mt_perturb_bt_qg.jsonl
 create mode 100644 dataNoPerturb/qa_bt.jsonl
 create mode 100644 dataNoPerturb/qa_src.jsonl
 create mode 100644 dataNoPerturb/src_mt_bt.jsonl
 create mode 100644 dataNoPerturb/src_mt_bt_qg.jsonl
 create mode 100644 dataNoPerturb/src_multillm_clean.jsonl
 create mode 100644 results/sbert_src_vs_bt.csv
 create mode 100644 sbert.py
 create mode 100644 src_multillm_clean.jsonl


In [30]:
INSERIRE TOKEN

In [31]:
!git push origin main


Enumerating objects: 25, done.
Counting objects: 100% (25/25), done.
Delta compression using up to 2 threads
Compressing objects: 100% (17/17), done.
Writing objects: 100% (18/18), 6.98 KiB | 3.49 MiB/s, done.
Total 18 (delta 6), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (6/6), completed with 2 local objects.
To https://github.com/AlessiaCicca/ASKQE-Hallucination.git
   d44a790..6a4861a  main -> main


In [32]:
!git status


On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


CAMBIA NOME FILE

with open("data/src_mt_bt.jsonl", 'r', encoding="utf-8") as f_in

HO IL FILE IN 'src' e non 'en':

sentence = data.get('src', None)


rimuovi il to device




Impostare esplicitamente il pad_token

Subito dopo aver caricato tokenizer e model:

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id





---

USANDO UN MODELLO LLM PER TRADUZIONE CON PARAMETRI SCARSI E SENZA FARE POI LE PERTUBAZIONI


In [1]:
!git clone https://github.com/AlessiaCicca/ASKQE-Hallucination.git

Cloning into 'ASKQE-Hallucination'...
remote: Enumerating objects: 107, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 107 (delta 36), reused 62 (delta 22), pack-reused 0 (from 0)
Receiving objects: 100% (107/107), 42.99 KiB | 3.31 MiB/s, done.
Resolving deltas: 100% (36/36), done.


In [2]:
!rm -rf /content/ASKQE-Hallucination/data

In [4]:
LOGIN HUGGING FACE

In [6]:
# ============================================================
# MULTI-LLM EN → IT TRANSLATION (HF) — ROBUST & CLEAN
# ============================================================
# pip install transformers torch accelerate bitsandbytes sentencepiece

import json
import os
import torch
import re
from transformers import AutoTokenizer, AutoModelForCausalLM

# -------------------------
# CONFIG
# -------------------------
INPUT_PATH = "/content/ASKQE-Hallucination/data.json"
OUTPUT_PATH = "/content/ASKQE-Hallucination/dataNoPerturb/src_multillm_clean.jsonl"

MODELS = {
    "qwen": "Qwen/Qwen2.5-3B-Instruct",
    "mistral": "mistralai/Mistral-7B-Instruct-v0.2",
    "gemma": "google/gemma-2b-it"
}

GEN_KWARGS = dict(
    max_new_tokens=1000,
    do_sample=True,
    temperature=1.2,      # induce inferenze spurie
    top_p=1,
    repetition_penalty=1.05

)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"[INFO] Using device: {device}")

os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

# -------------------------
# LOAD DATA
# -------------------------
def load_data(path):
    with open(path, "r", encoding="utf-8") as f:
        first = f.read(1)
        f.seek(0)
        if first == "[":
            return json.load(f)
        return [json.loads(line) for line in f if line.strip()]

data = load_data(INPUT_PATH)
print(f"[OK] Loaded {len(data)} samples")

# -------------------------
# PROMPT (MINIMAL & ROBUST)
# -------------------------
def build_prompt(text):
    return f"""Translate from English to Italian.

English:
{text}

Italian:"""

# -------------------------
# CLEANING
# -------------------------
def clean_translation(text):
    text = text.strip()

    text = re.sub(
        r"^(italian|italiano|italian translation|italian sentence)\s*:?",
        "",
        text,
        flags=re.IGNORECASE
    ).strip()

    text = text.split("\n")[0].strip()
    return text

# -------------------------
# RUN MODELS
# -------------------------
for tag, model_name in MODELS.items():
    print(f"\n[LOAD MODEL] {model_name}")

    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        torch_dtype=torch.float16 if device == "cuda" else torch.float32
    )
    model.eval()

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    for i, ex in enumerate(data, 1):
        src = ex.get("src")
        if not src:
            continue

        prompt = build_prompt(src)
        inputs = tokenizer(
            prompt,
            return_tensors="pt",
            truncation=True,
            max_length=512
        ).to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                **GEN_KWARGS,
                eos_token_id=tokenizer.eos_token_id
            )

        decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

        if "Italian:" in decoded:
            text = decoded.split("Italian:", 1)[1]
        else:
            text = decoded

        text = clean_translation(text)

        # fallback di sicurezza
        if len(text.split()) < 3:
            text = ""

        ex[f"mt_{tag}"] = text

        if i % 50 == 0:
            print(f"[{tag.upper()}] {i}/{len(data)}")

    del model
    torch.cuda.empty_cache()

# -------------------------
# SAVE OUTPUT
# -------------------------
with open(OUTPUT_PATH, "w", encoding="utf-8") as f:
    for ex in data:
        f.write(json.dumps(ex, ensure_ascii=False) + "\n")

print("\n✅ COMPLETED — MULTI-LLM CLEAN TRANSLATION")
print(f"📄 Output saved to: {OUTPUT_PATH}")


[INFO] Using device: cuda
[OK] Loaded 3 samples

[LOAD MODEL] Qwen/Qwen2.5-3B-Instruct


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


[LOAD MODEL] mistralai/Mistral-7B-Instruct-v0.2


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



[LOAD MODEL] google/gemma-2b-it


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


✅ COMPLETED — MULTI-LLM CLEAN TRANSLATION
📄 Output saved to: /content/ASKQE-Hallucination/src_multillm_clean.jsonl


OUTPUT TYPE:

{"id": "ex3", "src": "School closures in the city of Oita, Japan, were found to have successfully decreased the number of infected students at the peak of infection; however closing schools was not found to have significantly decreased the total number of infected students.", "mt_qwen": "È stato riscontrato che gli sblocci delle scuole nella città di Oita, Giappone, hanno riuscito a ridurre il numero degli studenti infettati al picco dell'infezione; tuttavia, l'apertura delle scuole non è stata trovata a ridurre significativamente il totale degli studenti infettati.", "mt_mistral": "Le chiusure scolastiche nella città di Oita, Giappone, sono state ritrovate efficienti nel ridurre il numero di studenti infetti in cima all'epidemia; tuttavia la chiusura delle scuole non è stata trovata significativamente utila per diminuire il numero totale di studenti infettati.", "mt_gemma": "I have been informed that school closures in the city of Oita, Japan, have been successfully reduced - at their peak - in the number of infected students; however, closing schools has not significantly reduced the total number of infected students."}


In [8]:
import json
import os
import torch
from transformers import MarianMTModel, MarianTokenizer,AutoModelForSeq2SeqLM
import gc
import re

# =========================
# DEVICE
# =========================
default_device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"[INFO] Using device: {default_device}")

# =========================
# MODELS (SOLO 2 MODELLI)
# =========================

BT_MODELS = {
    "bt1": "Helsinki-NLP/opus-mt-it-en",          # Standard IT→EN
    "bt2": "Helsinki-NLP/opus-mt-tc-big-it-en"   # Big IT→EN
}

# =========================
# LOAD MODEL
# =========================
def load_model(model_name):
    print(f"[LOAD] {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
    return tokenizer, model

def free_model(model, tokenizer):
    if model is not None:
        del model
    if tokenizer is not None:
        del tokenizer
    torch.cuda.empty_cache()
    gc.collect()

# =========================
# TRANSLATION FUNCTION
# =========================
def translate_marian(text, tokenizer, model):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    ).to(device)

    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=256)

    return tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

# =========================
# LOAD DATA
# =========================
INPUT_PATH = "/content/ASKQE-Hallucination/dataNoPerturb/src_multillm_clean.jsonl"   # json o jsonl
OUTPUT_PATH = "/content/ASKQE-Hallucination/dataNoPerturb/src_mt_bt.jsonl"

os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

def load_data(path):
    print(f"[STEP] Loading dataset: {path}")
    with open(path, "r", encoding="utf-8") as f:
        first_char = f.read(1)
        f.seek(0)
        if first_char == "[":
            data = json.load(f)
            print(f"[OK] JSON list with {len(data)} samples")
            return data
        else:
            data = [json.loads(line) for line in f if line.strip()]
            print(f"[OK] JSONL with {len(data)} samples")
            return data

data_list = load_data(INPUT_PATH)


# =========================
# PHASE 2: BACK-TRANSLATION (IT → EN)
# =========================
print("\n" + "="*70)
print("PHASE 2: BACK-TRANSLATION (IT → EN) - 2 MODELS × 2 MT = 4")
print("="*70 + "\n")
for bt_key_outer, bt_model_name_outer in BT_MODELS.items():
    print(f"\n[BT] Loading {bt_key_outer}: {bt_model_name_outer.split('/')[-1]}")
    bt_tok_outer, bt_model_outer = load_model(bt_model_name_outer)

    # bt1 → pert_mt1, bt2 → pert_mt2

    for idx, ex in enumerate(data_list, start=1):

      # tutte le chiavi che iniziano con "mt"
      mt_keys = [k for k in ex.keys() if k.startswith("mt")]

      for mt_key in mt_keys:
          input_text = ex[mt_key]

          # es: bt1_mt_qwen, bt2_mt_mistral, ecc.
          out_key = f"{bt_key_outer}_{mt_key}"

          ex[out_key] = translate_marian(
              input_text,
              bt_tok_outer,
              bt_model_outer
          )

          print(f"  [{idx}/{len(data_list)}] {out_key}: {ex[out_key][:55]}...")

    print(f"[CLEANUP] Freeing {bt_key_outer}")
    free_model(bt_model_outer, bt_tok_outer)


# =========================
# SAVE
# =========================
print("\n" + "="*70)
print("SAVING RESULTS")
print("="*70 + "\n")

with open(OUTPUT_PATH, "w", encoding="utf-8") as fout:
    for ex in data_list:
        fout.write(json.dumps(ex, ensure_ascii=False) + "\n")

print("✅ PIPELINE COMPLETED SUCCESSFULLY!")
print(f"📄 Output saved to: {OUTPUT_PATH}")
print(f"📊 Total samples: {len(data_list)}")
print("📋 Fields per sample: src + 2 MT (IT) + 4 BT (EN) = 7 fields total")
print("\nModels used:")
print("  BT1: Helsinki-NLP/opus-mt-it-en")
print("  BT2: Helsinki-NLP/opus-mt-tc-big-it-en")


[INFO] Using device: cuda
[STEP] Loading dataset: /content/ASKQE-Hallucination/dataNoPerturb/src_multillm_clean.jsonl
[OK] JSONL with 3 samples

PHASE 2: BACK-TRANSLATION (IT → EN) - 2 MODELS × 2 MT = 4


[BT] Loading bt1: opus-mt-it-en
[LOAD] Helsinki-NLP/opus-mt-it-en


/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/344M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/344M [00:00<?, ?B/s]

...

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/576M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/576M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

  [1/3] bt2_mt_qwen: Articles published by China and Italy suggest risks ass...
  [1/3] bt2_mt_mistral: Reports published by China and Italy suggest that risk ...
  [1/3] bt2_mt_gemma: Public reported islands from China and Italy have sugge...
  [2/3] bt2_mt_qwen: In some homes, the removal of schools after a period of...
  [2/3] bt2_mt_mistral: In some cases, the reopening of schools after a period ...
  [2/3] bt2_mt_gemma: In some cases, the reopening of schools after a period ...
  [3/3] bt2_mt_qwen: It has been found that school breakdowns in the city of...
  [3/3] bt2_mt_mistral: School closures in the city of Oita, Japan were found t...
  [3/3] bt2_mt_gemma: I have been informed that school closures in the city o...
[CLEANUP] Freeing bt2

SAVING RESULTS

✅ PIPELINE COMPLETED SUCCESSFULLY!
📄 Output saved to: /content/ASKQE-Hallucination/dataNoPerturb/src_mt_bt.jsonl
📊 Total samples: 3
📋 Fields per sample: src + 2 MT (IT) + 4 BT (EN) = 7 fields total

Models used:
  BT1: Helsinki-N

In [10]:
!python -u /content/ASKQE-Hallucination/QG/llama-8b.py \
  --output_path /content/ASKQE-Hallucination/dataNoPerturb/src_mt_bt_qg.jsonl \
  --prompt vanilla

`torch_dtype` is deprecated! Use `dtype` instead!
2025-12-22 20:11:57.861736: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766434317.888712   10719 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766434317.896510   10719 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766434317.926466   10719 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766434317.926490   10719 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766434317.926494   10719

In [11]:
#QA su SRC
!python ASKQE-Hallucination/QA/llama-8b.py \
  --sentence_type src \
  --input_path /content/ASKQE-Hallucination/dataNoPerturb/src_mt_bt_qg.jsonl \
  --output_path /content/ASKQE-Hallucination/dataNoPerturb/qa_src.jsonl


[INFO] Using device: cuda
`torch_dtype` is deprecated! Use `dtype` instead!
2025-12-22 20:14:31.266452: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766434471.306938   11363 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766434471.316777   11363 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766434471.340028   11363 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766434471.340075   11363 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00

In [12]:
#QA su SRC
!python ASKQE-Hallucination/QA/llama-8b.py \
  --sentence_type bt \
  --input_path /content/ASKQE-Hallucination/dataNoPerturb/src_mt_bt_qg.jsonl \
  --output_path /content/ASKQE-Hallucination/dataNoPerturb/qa_bt.jsonl


[INFO] Using device: cuda
`torch_dtype` is deprecated! Use `dtype` instead!
2025-12-22 20:16:15.062887: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766434575.086376   11827 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766434575.091771   11827 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766434575.106874   11827 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766434575.106896   11827 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00

In [13]:
import json

def ucr_hallucination(answers_src, answers_bt, no_answer="No Answer"):
    assert len(answers_src) == len(answers_bt)

    hallucinated = 0
    for a_src, a_bt in zip(answers_src, answers_bt):
        if a_src.strip() == no_answer and a_bt.strip() != no_answer:
            print(a_src)
            print(a_bt)
            hallucinated += 1

    return hallucinated / max(1, len(answers_src))


INPUT_PATH_src = "/content/ASKQE-Hallucination/dataNoPerturb/qa_src.jsonl"
INPUT_PATH_bt = "/content/ASKQE-Hallucination/dataNoPerturb/qa_bt.jsonl"


def load_data(path):
    print(f"[STEP] Loading dataset: {path}")
    with open(path, "r", encoding="utf-8") as f:
        first_char = f.read(1)
        f.seek(0)
        if first_char == "[":
            data = json.load(f)
            print(f"[OK] JSON list with {len(data)} samples")
            return data
        else:
            data = [json.loads(line) for line in f if line.strip()]
            print(f"[OK] JSONL with {len(data)} samples")
            return data

src = load_data(INPUT_PATH_src)
bt = load_data(INPUT_PATH_bt)

# Aggregate all answers into flat lists
all_answers_src_bt1 = []
all_answers_bt1 = []

for src_item, bt_item in zip(src, bt):
    # Extract and parse answers_src_bt1
    if "answers_src_bt1" in src_item and src_item["answers_src_bt1"]:
        # The value is a list containing a JSON string of a list
        json_list_str = src_item["answers_src_bt1"][0]
        try:
            parsed_list = json.loads(json_list_str)
            all_answers_src_bt1.extend(parsed_list)
        except json.JSONDecodeError:
            print(f"Warning: Could not parse answers_src_bt1 for example ID {src_item.get('id', 'N/A')}")

    # Extract and parse answers_bt1
    if "answers_bt1" in bt_item and bt_item["answers_bt1"]:
        # The value is a list containing a JSON string of a list
        json_list_str = bt_item["answers_bt1"][0]
        try:
            parsed_list = json.loads(json_list_str)
            all_answers_bt1.extend(parsed_list)
        except json.JSONDecodeError:
            print(f"Warning: Could not parse answers_bt1 for example ID {bt_item.get('id', 'N/A')}")


ucr_bt1 = ucr_hallucination(
    all_answers_src_bt1,
    all_answers_bt1
)
print(ucr_bt1)


[STEP] Loading dataset: /content/ASKQE-Hallucination/dataNoPerturb/qa_src.jsonl
[OK] JSONL with 3 samples
[STEP] Loading dataset: /content/ASKQE-Hallucination/dataNoPerturb/qa_bt.jsonl
[OK] JSONL with 3 samples
0.0


In [16]:
!python /content/ASKQE-Hallucination/sbert.py

2025-12-22 20:31:13.800935: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766435473.822888   15610 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766435473.829536   15610 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766435473.845931   15610 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766435473.845959   15610 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766435473.845963   15610 computation_placer.cc:177] computation placer alr

In [18]:
%cd /content/ASKQE-Hallucination
!git status


/content/ASKQE-Hallucination
On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   QA/__pycache__/prompt.cpython-312.pyc
	modified:   QG/__pycache__/prompt.cpython-312.pyc
	modified:   QG/llama-8b.py
	deleted:    data/qa_bt.jsonl
	deleted:    data/qa_src.jsonl
	deleted:    data/src_mt.jsonl
	deleted:    data/src_mt_perturb.jsonl
	deleted:    data/src_mt_perturb_bt.jsonl
	deleted:    data/src_mt_perturb_bt_qg.jsonl

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	dataNoPerturb/
	results/
	sbert.py
	src_multillm_clean.jsonl

no changes added to commit (use "git add" and/or "git commit -a")


In [20]:
!git config --global user.email "alessia.ciccaglione02@gmail.com"
!git config --global user.name "AlessiaCicca"


In [22]:
INSERIRE TOKEN

In [23]:
!echo "data/" >> .gitignore
!git add .gitignore
!git commit -m "No Pertubation Code, Transaltion with LLM"
!git push


[main d44a790] No Pertubation Code, Transaltion with LLM
 1 file changed, 1 insertion(+)
Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 2 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (6/6), 594 bytes | 594.00 KiB/s, done.
Total 6 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 1 local object.
To https://github.com/AlessiaCicca/ASKQE-Hallucination.git
   547e2e9..d44a790  main -> main
